In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [51]:
# we import useful libraries
import time
from robot_properties_nyu_finger.wrapper import NYUFingerDoubleRobot
from bullet_utils.env import BulletEnv
import numpy as np
import pybullet
import pinocchio as pin

In [8]:
env = BulletEnv(pybullet.DIRECT)
robot = NYUFingerDoubleRobot()
env.add_robot(robot)

In [25]:
q, v = robot.get_state()

In [53]:
model = robot.pin_robot.model
data = robot.pin_robot.data

In [54]:
ee0_id = model.getFrameId("finger0_lower_to_tip_joint")
ee1_id = model.getFrameId("finger1_lower_to_tip_joint")

In [68]:
pin.forwardKinematics(model, data, q, v)
ee0_pose = pin.updateFramePlacement(model, data, ee0_id)
ee1_pose = pin.updateFramePlacement(model, data, ee1_id)

In [90]:
body_jacobian_ee0 = pin.computeFrameJacobian(model, data, q, ee0_id)
body_jacobian_ee1 = pin.computeFrameJacobian(model, data, q, ee1_id)

In [91]:
R_ee0 = ee0_pose.rotation
R_ee1 = ee1_pose.rotation

In [92]:
Ad0 = np.zeros((6, 6))
Ad0[:3, :3] = R_ee0
Ad0[3:, 3:] = R_ee0

Ad1 = np.zeros((6, 6))
Ad1[:3, :3] = R_ee1
Ad1[3:, 3:] = R_ee1

In [93]:
oriented_jacobian_ee0 = Ad0 @ body_jacobian_ee0
oriented_jacobian_ee1 = Ad1 @ body_jacobian_ee1

In [100]:
Jov0 = oriented_jacobian_ee0[:3, :3]
Jov1 = oriented_jacobian_ee1[:3, :3]


In [40]:
for i in range(20):
    print(robot.pin_robot.model.frames[i].name)

universe
root_joint
base_link
base_to_finger0
finger0_base_link
finger0_base_to_upper_joint
finger0_upper_link
finger0_upper_to_middle_joint
finger0_middle_link
finger0_middle_to_lower_joint
finger0_lower_link
finger0_lower_to_tip_joint
finger0_tip_link
base_to_finger1
finger1_base_link
finger1_base_to_upper_joint
finger1_upper_link
finger1_upper_to_middle_joint
finger1_middle_link
finger1_middle_to_lower_joint


In [38]:
robot.pin_robot.data.oMf?

In [37]:
robot.pin_robot.model.names[6]

'finger1_middle_to_lower_joint'

In [3]:
# ! Create a Pybullet simulation environment before any robots !
env = BulletEnv()

# Create a robot instance. This adds the robot to the simulator as well.
robot = NYUFingerDoubleRobot()

# Add the robot to the env to update the internal structure of the robot
# at every simulation steps.
env.add_robot(robot)

In [5]:
robot.pin_robot.computeJointJacobian?

In [1]:
def velocity_control_circle(center, radius, run_time=5.0, plotting=False):
    '''
    draw a circle in the xy plane
    center (list): center of the circle
    radius (float): radius of the circle
    run_time (int): runtime in seconds
    return:
    '''
    # check len of center
    assert len(center) == 2, "center should include 2 values"
    # calculate desired position and velocity
    def get_pv(time, omega):
        '''
        return the position and velocity at given time and omega
        '''
        p_ref = np.array([center[0] + radius * np.cos(omega * time),
                          center[1] + radius * np.sin(omega * time),
                          0.])
        dp_ref = np.array([-radius * omega * np.sin(omega * time),
                           radius * omega * np.cos(omega * time),
                           0.])
        return p_ref, dp_ref

    # initialize robot
    # TODO: might not want to reset state
    q0 = np.ones(robot.nq)
    dq0 = np.zeros(robot.nv)
    robot.reset_state(q0, dq0)
    
    # set p, d values
    D = np.array([0.3, 0.3, 0.3])
    
    # the P gain for the velocity control (cf. equations above)
    gain = 2.
    
    # 1 ms for each step
    dt = 0.001
    num_steps = int(run_time/dt)
    
    if plotting:
        # we store information
        # here we create some arrays that we use to store data generated during the control loop
        measured_positions = np.zeros([num_steps,3]) # will store the measured position
        measured_velocities = np.zeros_like(measured_positions) # will store the measure velocities

        desired_torques = np.zeros_like(measured_positions) # will store the commands we send to the robot
        desired_positions = np.zeros_like(measured_positions) # will store the desired positions we use in the PD controller
        desired_velocities = np.zeros_like(measured_positions) # will store the desired velocities

        reference_foot_positions = np.zeros([num_steps, 3])
        reference_foot_velocities = np.zeros([num_steps, 3])

        simulation_time = np.zeros([num_steps]) # will store the running time

        x_pos = np.zeros([num_steps]) # will store the x position of the foot (as computed by Forw. Kin.)
        y_pos = np.zeros([num_steps]) # will store the y position of the foot (as computed by Forw. Kin.)
        z_pos = np.zeros([num_steps]) # will store the z position of the foot (as computed by Forw. Kin.)

        x_vel = np.zeros([num_steps]) # will store the x velocity of the foot
        y_vel = np.zeros([num_steps]) # will store the y velocity of the foot
        z_vel = np.zeros([num_steps]) # will store the z velocity of the foot
    
    
    # now we can enter the main control loop (each loop is 1 control cycle)
    for i in range(num_steps):

        # we get the position and velocities of the joints and save them
        q, dq = robot.get_state()
        
        if plotting:
            measured_positions[i,:] = q
            measured_velocities[i,:] = dq
            # get the current time and store it
            simulation_time[i] = dt * i
            
        '''
        1. Compute forward kinematics and get end-effector position
        2. Compute Jacobians. Note: in which frame you should compute it. If you computed the body Jacobian
        how do you convert it to the desired frame?
        '''

        # save the current position of the foot using the FK function    
        kin.update_kinematics(q)
        if plotting:
            x_pos[i] = kin.TS_F[0,3]
            y_pos[i] = kin.TS_F[1,3]
            z_pos[i] = kin.TS_F[2,3]

        ####FILLL HERE THE VELOCITY CONTROLLER
        x_measured = np.array([kin.TS_F[0,3],
                               kin.TS_F[1,3],
                               kin.TS_F[2,3]])
        
        x_ref, dx_ref = get_pv(dt * i, np.pi)

        if plotting:
            reference_foot_positions[i] = x_ref
            reference_foot_velocities[i] = dx_ref

        Jov = kin.orientedJ[3:6, :]
        
        if plotting:
            x_vel[i], y_vel[i], z_vel[i] = Jov @ dq

        if np.abs(q[2] < 0.05):
            Jov_inv = np.linalg.pinv(Jov + 1e-4*np.eye(3))
        else:
            Jov_inv = np.linalg.pinv(Jov)

        dx_des = dx_ref + gain * (x_ref - x_measured)
        dq_des = Jov_inv @ dx_des

        if plotting:
            desired_positions[i,:] = q # there is no desired joint position so we just store the current one
            desired_velocities[i,:] = dq_des

        d_error = dq_des - dq # the velocity error for all the joints

        # we compute the desired torques as a PD controller
        joint_torques = D * d_error
        if plotting:
            desired_torques[i,:] = joint_torques

        # we send them to the robot and do one simulation step
        robot.send_joint_command(joint_torques)
        env.step(sleep=True)
        
# todo: test
velocity_control_circle([0.3, -0.2], 0.05)

NameError: name 'np' is not defined

In [6]:
# TODO: add collision detection

# Some control.
tau = np.zeros(robot.nv)

# Reset the robot to some initial state.
q0 = 0.5 * np.ones(robot.nq)
dq0 = np.zeros(robot.nv)
robot.reset_state(q0, dq0)

# Run the simulator for 2000 steps
for i in range(2000):
    # TODO: Implement a controller here.
    robot.send_joint_command(tau)

    # Step the simulator.
    env.step(
        sleep=True
    )  # You can sleep here if you want to slow down the replay

# Read the final state and forces after the stepping.
q, dq = robot.get_state()
active_eff, forces = robot.get_force()
print("q", q)
print("dq", dq)
print("active eff", active_eff)
print("forces", forces)

q [-0.82513369  0.64415761  0.50698405 -0.82513369  0.64415761  0.50698405]
dq [-1.49455941 -2.99487857  2.51870468 -1.49455941 -2.99487857  2.51870468]
active eff []
forces []
